# Import Data

In [13]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [2]:
# District Summary
# Total number of schools and students
Number_of_schools=len(school_data["school_name"].unique())
Number_of_students=student_data["student_name"].count()

# Total budget of all the schools
Total_budget=school_data["budget"].sum()

# Average math score and reading score of all the students
Average_Math_Score=school_data_complete["math_score"].mean()
Average_Math_Score=round(Average_Math_Score,6)
Average_Reading_Score=school_data_complete["reading_score"].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
Students_with_passing_math=len(school_data_complete.loc[school_data_complete["math_score"]>=70])
passing_math_percent=Students_with_passing_math/Number_of_students

#Calculate the percentage of students with a passing reading score (70 or greater)
Students_with_passing_reading=len(school_data_complete.loc[school_data_complete["reading_score"]>=70])
passing_reading_percent=Students_with_passing_reading/Number_of_students

#Calculate the percentage of students who passed math and reading (% Overall Passing)
overall_passing=len(school_data_complete.loc[(school_data_complete["math_score"]>=70)
                                             &(school_data_complete["reading_score"]>=70)])
overall_passing_percent=overall_passing/Number_of_students

#Create a dataframe to hold the above results
district_summary_df=pd.DataFrame({"Total Schools":[Number_of_schools],
                        "Total Students":[Number_of_students],
                        "Total Budget":[Total_budget],
                        "Average Math Score":[Average_Math_Score],
                        "Average Reading Score":[Average_Reading_Score],
                        "% Passing Math":[passing_math_percent],
                        "% Passing Reading":[passing_reading_percent],
                        "% Overall Passing":[overall_passing_percent]})

#Optional: give the displayed data cleaner formatting
district_summary_df["Total Students"]=district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"]=district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["% Passing Math"]=round(district_summary_df["% Passing Math"]*100,6)
district_summary_df["% Passing Reading"]=round(district_summary_df["% Passing Reading"]*100,6)
district_summary_df["% Overall Passing"]=round(district_summary_df["% Overall Passing"]*100,6)
district_summary_df



,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary

In [3]:
# school summary:School Name, School Type, Total Students, Total School Budget
# Per Student Budget,Average Math Score,Average Reading Score

school_data_part=school_data_complete[['student_name','school_name',
       'reading_score', 'math_score','type', 'size', 'budget']]

school_group=school_data_part.groupby(["school_name","type","size","budget"])
school_summary_df=school_group.mean().reset_index()
school_summary_df["Per_Student_Budget"]=school_summary_df['budget'] / school_summary_df['size']
school_summary_df


,school_name,type,size,budget,reading_score,math_score,Per_Student_Budget
0,Bailey High School,District,4976,3124928,81.033963,77.048432,628.0
1,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0
2,Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0
3,Ford High School,District,2739,1763916,80.746258,77.102592,644.0
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0
5,Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0
6,Holden High School,Charter,427,248087,83.814988,83.803279,581.0
7,Huang High School,District,2917,1910635,81.182722,76.629414,655.0
8,Johnson High School,District,4761,3094650,80.966394,77.072464,650.0
9,Pena High School,Charter,962,585858,84.044699,83.839917,609.0


In [4]:
# number of students with passing math score
math_passing_df=school_group.agg(math_passing=('math_score', lambda x: (x >= 70).sum())).reset_index()
# merge to summary dataframe
school_summary_df=school_summary_df.merge(math_passing_df)

# number of students with passing reading score
reading_passing_df=school_group.agg(reading_passing=('reading_score', lambda x: (x >= 70).sum())).reset_index()
#merge to summary dataframe
school_summary_df=school_summary_df.merge(reading_passing_df)

#percent of students with passing math score
math_passing_percent=school_summary_df["math_passing"]/school_summary_df["size"]
#percent of students with passing reading score
reading_passing_percent=school_summary_df["reading_passing"]/school_summary_df["size"]

# add new column named math_passing percent and reading_passing_percent
school_summary_df["math_passing_percent"]=math_passing_percent*100
school_summary_df["reading_passing_percent"]=reading_passing_percent*100
school_summary_df

,school_name,type,size,budget,reading_score,math_score,Per_Student_Budget,math_passing,reading_passing,math_passing_percent,reading_passing_percent
0,Bailey High School,District,4976,3124928,81.033963,77.048432,628.0,3318,4077,66.680064,81.933280
1,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0,1749,1803,94.133477,97.039828
2,Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0,1946,2381,65.988471,80.739234
3,Ford High School,District,2739,1763916,80.746258,77.102592,644.0,1871,2172,68.309602,79.299014
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0,1371,1426,93.392371,97.138965
5,Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0,3094,3748,66.752967,80.862999
6,Holden High School,Charter,427,248087,83.814988,83.803279,581.0,395,411,92.505855,96.252927
7,Huang High School,District,2917,1910635,81.182722,76.629414,655.0,1916,2372,65.683922,81.316421
8,Johnson High School,District,4761,3094650,80.966394,77.072464,650.0,3145,3867,66.057551,81.222432
9,Pena High School,Charter,962,585858,84.044699,83.839917,609.0,910,923,94.594595,95.945946


In [5]:
# set up a new DataFrame with columns from the original complete dataframe 
school_data_score_df=school_data_complete[['school_name',"student_name",'reading_score','math_score']]

#filter out students with overall passing scores from the original complete dataframe
overall_passing_df=school_data_score_df.loc[(school_data_score_df["reading_score"]>=70)&
                                               (school_data_score_df["math_score"]>=70)]

# groupby school names and get the number of students with overall passing for each school
overall_passing_df=overall_passing_df.groupby("school_name").count()

# reset index
overall_passing_df=overall_passing_df.reset_index()

# add one column "overal_passing" to school summary dataframe 
school_summary_df["overall_passing"]=overall_passing_df["student_name"]

# get the overall_passing_percent
school_summary_df["overall_passing_percent"]=(school_summary_df["overall_passing"]/school_summary_df["size"])*100

# get the columns needed
school_summary_df=school_summary_df[['school_name', 'type', 'size', 'budget', 
                                     'Per_Student_Budget', "reading_score", 
                                     "math_score", 'math_passing_percent',
                                     'reading_passing_percent','overall_passing_percent']]
school_summary_df.head()

# set index and rename columns
school_summary_df.set_index("school_name")
school_summary_df=school_summary_df.rename(columns={'type':"School_Type", 
                                  'size':"Total Students", 
                                  'budget':"Total School Budget", 
                                  "math_score":"Average Math Score",
                                  "reading_score":"Average Reading Score",
                                  'math_passing_percent':"% Passing Math",
                                  'reading_passing_percent':"% Passing Reading",
                                  'overall_passing_percent':"% Overall Passing"})


# format data
school_summary_df["Total School Budget"]=school_summary_df["Total School Budget"].map("${:,.2f}".format)
school_summary_df["Per_Student_Budget"]=school_summary_df["Per_Student_Budget"].map("${:,.2f}".format)
school_summary_df.head()

,school_name,School_Type,Total Students,Total School Budget,Per_Student_Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
0,Bailey High School,District,4976,"$3,124,928.00",$628.00,81.033963,77.048432,66.680064,81.933280,54.642283
1,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.975780,83.061895,94.133477,97.039828,91.334769
2,Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,65.988471,80.739234,53.204476
3,Ford High School,District,2739,"$1,763,916.00",$644.00,80.746258,77.102592,68.309602,79.299014,54.289887
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.816757,83.351499,93.392371,97.138965,90.599455


# Top Performing School

In [6]:
# Sort and display the top five performing schools by % overall passing
Top_Performing_df=school_summary_df.sort_values(by="% Overall Passing",ascending=False).head()
Top_Performing_df.set_index("school_name")

,School_Type,Total Students,Total School Budget,Per_Student_Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.975780,83.061895,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.848930,83.418349,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.816757,83.351499,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.989488,83.274201,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,84.044699,83.839917,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [7]:
# Sort and display the top five performing schools by % overall passing
Bottom_Performing_df=school_summary_df.sort_values(by="% Overall Passing",ascending=False).tail()
Bottom_Performing_df.set_index("school_name")

,School_Type,Total Students,Total School Budget,Per_Student_Budget,Average Reading Score,Average Math Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Johnson High School,District,4761,"$3,094,650.00",$650.00,80.966394,77.072464,66.057551,81.222432,53.539172
Hernandez High School,District,4635,"$3,022,020.00",$652.00,80.934412,77.289752,66.752967,80.862999,53.527508
Huang High School,District,2917,"$1,910,635.00",$655.00,81.182722,76.629414,65.683922,81.316421,53.513884
Figueroa High School,District,2949,"$1,884,411.00",$639.00,81.158020,76.711767,65.988471,80.739234,53.204476
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,80.744686,76.842711,66.366592,80.220055,52.988247


## Math Scores by Grade

In [8]:
# small dataframe with only school name, math score and grade
math_df=school_data_complete[["school_name","math_score","grade"]]

# get mean for 9th grade math for each school
grade_9th=math_df.loc[math_df["grade"]=="9th"]
grade_9th_df=grade_9th.groupby(["school_name"]).mean()

# get mean for 10th grade math for each school
grade_10th=math_df.loc[math_df["grade"]=="10th"]
grade_10th_df=grade_10th.groupby(["school_name"]).mean()

# get mean for 11th grade math for each school
grade_11th=math_df.loc[math_df["grade"]=="11th"]
grade_11th_df=grade_11th.groupby(["school_name"]).mean()


# get mean for 12th grade math for each school
grade_12th=math_df.loc[math_df["grade"]=="12th"]
grade_12th_df=grade_12th.groupby(["school_name"]).mean()

# merge all the above dataframes and rename the columns
combined_math_df=pd.merge(grade_9th_df,grade_10th_df,on="school_name")
combined_math_df=combined_math_df.rename(columns={"math_score_x":"9th","math_score_y":"10th"})
combined_math_df=pd.merge(combined_math_df,grade_11th_df,on="school_name")
combined_math_df=combined_math_df.rename(columns={"math_score":"11th"})
combined_math_df=pd.merge(combined_math_df,grade_12th_df,on="school_name")
combined_math_df=combined_math_df.rename(columns={"math_score":"12th"})
combined_math_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

In [9]:
# small dataframe with only school name, reading score and grade
reading_df=school_data_complete[["school_name","reading_score","grade"]]

# get mean for 9th grade reading for each school
grade_9th=reading_df.loc[reading_df["grade"]=="9th"]
grade_9th_df=grade_9th.groupby(["school_name"]).mean()

# get mean for 10th grade reading for each school
grade_10th=reading_df.loc[reading_df["grade"]=="10th"]
grade_10th_df=grade_10th.groupby(["school_name"]).mean()

# get mean for 11th grade reading for each school
grade_11th=reading_df.loc[reading_df["grade"]=="11th"]
grade_11th_df=grade_11th.groupby(["school_name"]).mean()


# get mean for 12th grade reading for each school
grade_12th=reading_df.loc[reading_df["grade"]=="12th"]
grade_12th_df=grade_12th.groupby(["school_name"]).mean()

# merge all the above dataframes and rename the columns
combined_reading_df=pd.merge(grade_9th_df,grade_10th_df,on="school_name")
combined_reading_df=combined_reading_df.rename(columns={"reading_score_x":"9th","reading_score_y":"10th"})
combined_reading_df=pd.merge(combined_reading_df,grade_11th_df,on="school_name")
combined_reading_df=combined_reading_df.rename(columns={"reading_score":"11th"})
combined_reading_df=pd.merge(combined_reading_df,grade_12th_df,on="school_name")
combined_reading_df=combined_reading_df.rename(columns={"reading_score":"12th"})
combined_reading_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

In [10]:
spending_df=school_summary_df[["Per_Student_Budget",
                               "school_name",
                               "Average Math Score",
                               "Average Reading Score",
                               "% Passing Math","% Passing Reading",
                               "% Overall Passing"]]

# format Per_Student_Budget to float
spending_df["Per_Student_Budget"]=spending_df["Per_Student_Budget"].str.replace("$"," ").astype("float")

# get the max and min value of Per_Student_Budget
print(spending_df["Per_Student_Budget"].max())
print(spending_df["Per_Student_Budget"].min())

#set up bins and labels
bins=[0,585,630,645,680]
group_names=["<$585","$585-630","$630-645","645-680"]

# use bins to group school spending
spending_df["Spending Ranges(Per Student)"]=pd.cut(spending_df["Per_Student_Budget"],bins=bins,labels=group_names)

# school performance based on average spending ranges(per student)
school_spending_df=round(spending_df.groupby("Spending Ranges(Per Student)").mean(),2)
school_spending_df=school_spending_df.drop(columns="Per_Student_Budget")
school_spending_df



655.0
578.0


<ipython-input-10-47562053f2db>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spending_df["Per_Student_Budget"]=spending_df["Per_Student_Budget"].str.replace("$"," ").astype("float")
<ipython-input-10-47562053f2db>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spending_df["Spending Ranges(Per Student)"]=pd.cut(spending_df["Per_Student_Budget"],bins=bins,labels=group_names)


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges(Per Student),,,,,
<$585,83.46,83.93,93.46,96.61,90.37
$585-630,81.90,83.16,87.13,92.72,81.42
$630-645,78.52,81.62,73.48,84.39,62.86
645-680,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

In [11]:
# size dataframe with columns needed
size_df=school_summary_df[["Total Students",
                           "Average Math Score",
                           "Average Reading Score",
                           "% Passing Math",
                           "% Passing Reading",
                           "% Overall Passing"]]

# get the max and min value of Per_Student_Budget
print(size_df["Total Students"].max())
print(size_df["Total Students"].min())

# set up bins and labels
bins=[0,1000,2000,5000]
group_names=["Small(<1000)","Medium(1000-2000)","Large(2000-5000)"]
size_df["School Size"]=pd.cut(size_df["Total Students"],bins=bins,labels=group_names)
size_df
# school performance based on school size
school_size_df=round(size_df.groupby("School Size").mean(),2)

school_size_df=school_size_df.drop(columns="Total Students")
school_size_df


4976
427


<ipython-input-11-580fff08adbe>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  size_df["School Size"]=pd.cut(size_df["Total Students"],bins=bins,labels=group_names)


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small(<1000),83.82,83.93,93.55,96.10,89.88
Medium(1000-2000),83.37,83.86,93.60,96.79,90.62
Large(2000-5000),77.75,81.34,69.96,82.77,58.29


## Scores by School Type

In [12]:
# size dataframe with columns needed
Type_df=school_summary_df[["School_Type",
                           "Average Math Score",
                           "Average Reading Score",
                           "% Passing Math",
                           "% Passing Reading",
                           "% Overall Passing"]]

# school performance based on school type
School_Type_df=round(Type_df.groupby("School_Type").mean(),2)
School_Type_df


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School_Type,,,,,
Charter,83.47,83.90,93.62,96.59,90.43
District,76.96,80.97,66.55,80.80,53.67
